## The forward and backward passes

In [3]:
# import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
import pickle,gzip,math,os,time,shutil,matplotlib as mpl, numpy as np
import tensorflow as tf
from pathlib import Path
# from torch import tensor
# from fastcore.test import test_close
# torch.manual_seed(42)
tf.random.set_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
# torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
# x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])
x_train, y_train, x_valid, y_valid = map(tf.constant, [x_train, y_train, x_valid, y_valid])

2023-11-11 11:20:48.946223: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-11 11:20:48.946239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: Hephaestus
2023-11-11 11:20:48.946242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: Hephaestus
2023-11-11 11:20:48.946283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-11 11:20:48.946291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.129.3
2023-11-11 11:20:48.946293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.129.3


## Foundations version

### Basic architecture

In [5]:
n,m = x_train.shape
# c = y_train.max()+1
c = tf.reduce_max(y_train)+1
n,m,c

(50000, 784, <tf.Tensor: shape=(), dtype=int64, numpy=10>)

In [8]:
# num hidden
nh = 50

In [10]:
# w1 = torch.randn(m,nh)
# b1 = torch.zeros(nh)
# w2 = torch.randn(nh,1)
# b2 = torch.zeros(1)

w1 = tf.random.normal((m, nh))
b1 = tf.zeros(nh)
w2 = tf.random.normal((nh, 1))
b2 = tf.zeros(1)

In [11]:
def lin(x, w, b): return x@w + b

In [12]:
t = lin(x_valid, w1, b1)
t.shape

TensorShape([10000, 50])

In [13]:
# def relu(x): return x.clamp_min(0.)
def relu(x): return tf.clip_by_value(x, clip_value_min=0., clip_value_max=np.inf) 

In [14]:
t = relu(t)
t

<tf.Tensor: shape=(10000, 50), dtype=float32, numpy=
array([[ 0.  ,  3.42,  0.11, ...,  0.  ,  0.  ,  0.1 ],
       [ 6.15,  0.  , 10.72, ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.34],
       ...,
       [ 0.  ,  4.  ,  8.84, ...,  7.57,  3.52,  5.29],
       [ 2.95,  0.  ,  0.  , ...,  0.  ,  0.22,  0.  ],
       [ 7.41,  2.55,  0.  , ...,  0.  ,  7.97,  0.  ]], dtype=float32)>

In [15]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [16]:
res = model(x_valid)
res.shape

TensorShape([10000, 1])

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [27]:
y_valid = tf.cast(y_valid, dtype=tf.float32)

In [28]:
res.shape,y_valid.shape

(TensorShape([10000, 1]), TensorShape([10000]))

In [29]:
(res-y_valid).shape

TensorShape([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [30]:
res[:,0].shape

TensorShape([10000])

In [31]:
# res.squeeze().shape
tf.squeeze(res).shape

TensorShape([10000])

In [32]:
(res[:,0]-y_valid).shape

TensorShape([10000])

In [38]:
# already cast y_valid before
# y_train,y_valid = y_train.float(),y_valid.float()
y_train = tf.cast(y_train, dtype=tf.float32)

preds = model(x_train)
preds.shape

TensorShape([50000, 1])

In [42]:
# def mse(output, targ): return (output[:,0]-targ).pow(2).mean()
def mse(output, targ): return tf.reduce_mean((output[:,0]-targ)**2)

In [43]:
mse(preds, y_train)

<tf.Tensor: shape=(), dtype=float32, numpy=1163.5092>

### Gradients and backward pass

In [46]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [47]:
diff(3*x**2+9, x)

6*x

In [55]:
# def lin_grad(inp, out, w, b):
#     # grad of matmul with respect to input
#     inp.g = out.g @ w.t()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0)

def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ tf.transpose(w)
    w.g = tf.reduce_sum(tf.expand_dims(inp, axis=-1) * tf.expand_dims(out.g, axis=1), axis=0) 
    b.g = tf.reduce_sum(out.g, axis=0)

In [56]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    # loss = diff.pow(2).mean()
    loss = tf.reduce_mean(diff**2)
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = tf.cast(l1>0, dtype=tf.float32) * l2.g
    lin_grad(inp, l1, w1, b1)

In [57]:
forward_and_backward(x_train, y_train)

In [59]:
# Save for testing against later
# def get_grad(x): return x.g.clone()
def get_grad(x): return tf.Variable(x.g)
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.

In [60]:
# def mkgrad(x): return x.clone().requires_grad_(True)
def mkgrad(x): return tf.Variable(x) # requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [61]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [63]:
# loss = forward(xt2, y_train)
# loss.backward()

In [65]:
with tf.GradientTape() as tape:
    # tape.watch(xt2)
    loss = forward(xt2, y_train)
    grads = tape.gradient(loss, xt2)

In [67]:
# for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

## Refactor model

### Layers as classes

In [68]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        # self.out = inp.clamp_min(0.)
        self.out = tf.clip_by_value(inp, clip_value_min=0., clip_value_max=np.inf)
        return self.out
    
    def backward(self): self.inp.g = tf.cast(self.inp>0, dtype=tf.float32) * self.out.g

In [69]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        # self.inp.g = self.out.g @ self.w.t()
        # self.w.g = self.inp.t() @ self.out.g
        # self.b.g = self.out.g.sum(0)

        self.inp.g = self.out.g @ tf.transpose(self.w)
        self.w.g = tf.transpose(self.inp) @ self.out.g
        self.b.g = tf.reduce_sum(self.out.g, axis=0)

In [71]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        # self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]
        self.inp.g = 2. * tf.expand_dims(tf.squeeze(self.inp) - self.targ, axis=-1) / self.targ.shape[0]

In [ ]:
# class Model():
#     def __init__(self, w1, b1, w2, b2):
#         self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
#         self.loss = Mse()
        
#     def __call__(self, x, targ):
#         for l in self.layers: x = l(x)
#         return self.loss(x, targ)
    
#     def backward(self):
#         self.loss.backward()
#         for l in reversed(self.layers): l.backward()

In [84]:
class Model(tf.keras.Model):
    def __init__(self, w1, b1, w2, b2):
        self._layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self._layers: x = l(x)
        return self.loss(x, targ)
    
    # def backward(self):
    #     self.loss.backward()
    #     for l in reversed(self.layers): l.backward()

    def train_step(self, x, targ):
        with tf.GradientTape() as tape:
            # tape.watch(x)
            loss = self(x, targ)
            grads = tape.gradient(loss, self._layers)
        return grads

In [85]:
model = Model(w1, b1, w2, b2)

In [86]:
loss = model(x_train, y_train)

In [87]:
# model.backward()
grads = model.train_step(x_train, y_train)

TypeError: Cannot convert the argument `type_value`: <__main__.Lin object at 0x7f3703481b70> to a TensorFlow DType.

In [ ]:
# test_close(w2g, w2.g, eps=0.01)
# test_close(b2g, b2.g, eps=0.01)
# test_close(w1g, w1.g, eps=0.01)
# test_close(b1g, b1.g, eps=0.01)
# test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])